In [9]:
import joblib
import pandas as pd

In [10]:
class MarketingPredictor:
    '''This class contains the logic for loading the trained timing model and product model,
    as well as the label encoder for product names. It defines which features each model uses
    and provides a predict method to get both the best email timing and the recommended product.'''

    def __init__(self, timing_model_path, product_model_path, label_encoder_path=None):
        # Load models and label encoder
        self.timing_model = joblib.load(timing_model_path)
        self.product_model = joblib.load(product_model_path)
        self.le = joblib.load(label_encoder_path) if label_encoder_path else None

        # Define the features required by the models
        self.product_features = ['has_peak', 'has_knife', 'has_pivot']
        self.timing_features = ['Order_Frequency', 'Average_Time_Between_Orders']

    def predict(self, customer_data):
        # Extract and prepare features for the timing model
        timing_input = {f: customer_data[f] for f in self.timing_features}
        timing_df = pd.DataFrame([timing_input])

        # Predict the best time to send the promotional email
        predicted_time = self.timing_model.predict(timing_df)[0]
        
        # Extract and prepare features for the product model
        product_input = {f: customer_data[f] for f in self.product_features}
        product_df = pd.DataFrame([product_input])

        # Predict the next product to advertise
        product_pred = self.product_model.predict(product_df)
        recommended_product = self.le.inverse_transform(product_pred)[0]

        return predicted_time, recommended_product
    


In [11]:
# Create a predictor instance
predictor = MarketingPredictor("rf_model.pkl", "product_model.pkl", "label_encoder.pkl")

# Define a new customer scenario with random features
new_customer = {
    "Order_Frequency": 3,
    "Average_Time_Between_Orders": 100,
    "has_peak": 1,
    "has_knife": 1,
    "has_pivot": 0
}
# Get predictions for this customer
predicted_time, recommended_product = predictor.predict(new_customer)

# Present the info in a sentence:
output_sentence = f"The best time to send a promotional email is approximately {int(predicted_time)} days after their last purchase, and the recommended product to advertise is '{recommended_product}'."

print(output_sentence)

The best time to send a promotional email is approximately 34 days after their last purchase, and the recommended product to advertise is 'pivot'.
